### Collisional-radiative model for Argon Plasma
#### ROM Basis

In [ ]:
import os
import sys
import numpy as np
import dill as pickle
import matplotlib.pyplot as plt

In [ ]:
sys.path.append("/home/zanardi/Codes/ML/ROMAr/romar/")
from romar import env
from romar import utils
from romar.systems import BoxAd
from romar import postproc as pp

In [ ]:
from factor_analyzer.rotator import Rotator, POSSIBLE_ROTATIONS

Set enviroment

In [ ]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

Set inputs

In [ ]:
paths = {
  # > ROM basis
  "basis": "/home/zanardi/Codes/ML/ROMAr/runs/run01/max_mom_2_GOOD/models_all/cobras/basis.p",
  # > Path to solutions folder
  "data": "/home/zanardi/Codes/ML/ROMAr/runs/run04/data/test/",
  # > Thermochemical database
  "dtb": "/home/zanardi/Codes/ML/ROMAr/romar/examples/database/",
  # > Output folder
  "out": "./visual_basis_figs/"
}
# Test cases names
cases = ["lrho_lT", "lrho_hT", "hrho_lT", "hrho_hT", "mrho_mT"]
# Number of ROM dimensions
rdim = 7
# Final time for integration
tend = 1e-3
# Labels for plotting
labels = {
  "Th": "$T_h$",
  "Te": "$T_e$",
  "Ar": "\\text{Ar}",
  "Arp": "\\text{Ar$^+$}",
  "em": "\\text{e$^-$}"
}

In [ ]:
os.makedirs(paths["out"], exist_ok=True)

Initialize 0D thermochemical system

In [ ]:
system = BoxAd(
  species={k: paths["dtb"] + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=paths["dtb"] + "/rates/kin_fit.p",
  rad_dtb=paths["dtb"] + "/rates/rad_fit.p",
  use_rad=True,
  use_proj=False,
  use_tables=False
)

In [ ]:
cmats = "/home/zanardi/Codes/ML/ROMAr/runs/run_04_cc/max_mom_2/models/cobras_wrong/cov_mats.p"
basis = "/home/zanardi/Codes/ML/ROMAr/runs/run_04_cc/max_mom_2/models/cobras_wrong/basis.p"

In [ ]:
with open(cmats, "rb") as file:
  cmats = pickle.load(file)
with open(basis, "rb") as file:
  basis = pickle.load(file)

In [ ]:
basis["s"]

In [ ]:
norm = np.linalg.norm(cmats["Y"], axis=0)
np.where(norm>8e2)

In [ ]:
basis["s"]

In [ ]:
norm = np.linalg.norm(cmats["Y"], axis=0)

In [ ]:
np.where(norm>8e2)

In [ ]:
import numpy as np

In [ ]:
path = "/home/zanardi/Codes/ML/ROMAr/runs/run_04_cc/data/train/case_0000.p"

In [ ]:
with open(path, "rb") as file:
  traj = pickle.load(file)

In [ ]:
x = traj["t"]
nt = len(x)

In [ ]:
n = 100
f = 0.333
x = np.arange(n)
s = np.round(f * n)
xi = np.array_split(x, s)
ii = np.asarray([np.take(xj, xj.size // 2) for xj in xi])
ii

In [ ]:
def sample_t0(
  nb_t: int,
  t0_perc: float
) -> np.ndarray:
  # Indices vector
  i = np.arange(nb_t)
  # Number of samples
  ns = np.round(t0_perc * nb_t)
  # Sample uniformly
  ii = np.array_split(i, ns)
  return np.asarray([j[j.size//2] for j in ii])

In [ ]:
sample_t0(100, 0.9)

In [ ]:
f = 0.25
s = int(1/f)
print(s)
i = np.arange(start=np.round(s/2), stop=len(x), step=s, dtype=int)
len(i), i

In [ ]:
f = 0.1
s = int(1/f)
print(s)
si = i[::s] + int(s/2)
si

In [ ]:
# take a value from the center
center = np.take(i, i.size // 2)
print(center)

In [ ]:
runtimes = [1, None, 2, None]
runtimes = [rt for rt in runtimes if (rt is not None)]
runtimes

Load ROM basis

In [ ]:
with open(paths["basis"], "rb") as file:
  basis = pickle.load(file)

In [ ]:
rot = Rotator(method="varimax")

In [ ]:
basis["phi"] = {k: rot.fit_transform(b) for k, b in basis["phi"].items()}
basis["psi"] = {k: rot.fit_transform(b) for k, b in basis["psi"].items()}
# basis["mask"][0] = 0.0
# basis["xref"][0] = 0.0
# basis["xscale"][0] = 1.0

In [ ]:
# with open(path, "wb") as file:
#   pickle.dump(basis, file)

In [ ]:
stop

In [ ]:
system.rom.build(
  phi=basis["phi"][rdim],
  psi=basis["psi"][rdim],
  **{k: basis[k] for k in ("mask", "xref", "xscale")}
)

> Check orthonormalization and singular values

In [ ]:
s = basis["s"]
phi = basis["phi"][rdim]
psi = basis["psi"][rdim]

In [ ]:
np.diag(phi.T @ psi), s

> Plot loadings

In [ ]:
for name in ("phi", "psi"):
  for i in range(rdim):
    nb = str(i+1)
    b = basis[name][rdim][:,i]
    pp.plot_dist_2d(
      x=np.arange(len(b)),
      y=b,
      labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % (name, nb)],
      scales=["linear", "linear"],
      markersize=1,
      figname=paths["out"] + f"/{name}_{nb.zfill(2)}",
      save=True,
      show=True
    )

Test ROM model

In [ ]:
testcases = {k: utils.load_case(filename=paths["data"] + "/case_" + k + ".p") for k in cases}

In [ ]:
for (k, testcase) in testcases.items():
  # Testcase
  print(k)
  print("="*20)
  # FOM solution
  t, y0, y, rho = [testcase[k] for k in ("t", "y0", "y", "rho")]
  it = np.argmin(np.abs(t-tend))
  t = t[:it]
  y = y[:,:it]
  system.mix.set_rho(rho)
  prim_fom = system.get_prim(y, clip=False)
  mom_fom = system.compute_mom(prim_fom[0])
  # ROM solution
  yrom, runtime = system.solve_rom(t, y0, rho)
  print("RUNTIME", runtime)
  prim_rom = system.get_prim(yrom, clip=False)
  mom_rom = system.compute_mom(prim_rom[0])
  nt = len(yrom.T)
  nt
  # Temperatures
  print("Th-Te")
  for i in range(2):
    plt.semilogx(t[:nt], prim_fom[1+i][:nt])
    plt.semilogx(t[:nt], prim_rom[1+i][:nt], ls="--", color=plt.gca().lines[-1].get_color(), lw=2.0)
  plt.show()
  plt.close()
  # Species
  for s in system.mix.species.values():
    print(s.name)
    for i in s.indices:
      plt.loglog(t[:nt], prim_fom[0][i,:nt])
      plt.loglog(t[:nt], prim_rom[0][i,:nt], ls="--", color=plt.gca().lines[-1].get_color(), lw=2.0)
    plt.show()
    plt.close()
  # Moments
  for s in system.mix.species.values():
    if (s.name != "em"):
      for m in range(2):
        print(s.name+f" - Moment: {m}")
        plt.semilogx(t[:nt], mom_fom[s.name][f"m{m}"][:nt])
        plt.semilogx(t[:nt], mom_rom[s.name][f"m{m}"][:nt], ls="--", color=plt.gca().lines[-1].get_color(), lw=2.0)
        plt.yscale("log" if (m == 0) else "linear")
        plt.show()
        plt.close()